In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, os
import matplotlib.dates as mdates
import pandas as pd
from sklearn.metrics import r2_score
import pandas as pd
from tqdm import tqdm


def MakeDataFromText(filepath:str):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234

In [ ]:
rssi = MakeDict([MakeDataFromText('CHANGING/91430/RSSI_91430.csv')])
temp = MakeDict([MakeDataFromText('CHANGING/91430/tags_91430.csv')])

In [ ]:
water =  pd.read_csv( 'CHANGING/91430/water temp_91430.txt', skiprows=1, sep=',', header=0 )
water.columns = [ 'Time', 'placeholder', 'TempC' ]
water = water.iloc[:3100]

water['Time']  = water['Time'].str.replace('\t', ' ')
water['TempC'] = water['TempC'].str.replace('\t', ' ')

n_ = []
for j in range(len(water['TempC'])):
    print(water['TempC'][j].split('.'))
    ones, dec = water['TempC'][j].split('.')
    n_.append(float(ones) + float(dec[0])*0.1)
water['TempC'] = n_

print(water['TempC'].tolist())

new = []
for i in range(len(water['Time'])):
    h,m,s = water['Time'][i].split(':')
    new.append ( datetime.datetime(1970, 2, 1, int(h[-2:])+1, int(m), int(s)))
water['Time'] = new

In [ ]:
fig, (ax1,ax2, ax3) = plt.subplots(3, 1, figsize=(16, 11), sharex=True)
ax1.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
ax2.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
ax3.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )

axs = [ax1, ax2, ax3]

TAGS = [ '460E', 'B642', '0F4B' ]

for i in range(3):
    for k,v in temp.items():
        if k[-4:]==TAGS[i]:
            st = sorted(temp[k].items())
            sr = sorted(rssi[k].items())
            dt, tt, dr = [j[1] for j in st], [j[0] for j in st], [j[1] for j in sr]

            new_dt, new_tt = [], []
            for p in range(len(dr)):
                if dr[p]==10:
                    new_dt.append(dt[p])
                    new_tt.append(tt[p])
            
            dt = new_dt
            tt = new_tt

            # Making Quadratic
            _t = [ (j-tt[0]).total_seconds() for j in tt ]
            model2 = np.poly1d( np.polyfit(_t, dt,  2) )
            strmodel2 = ", " + str(round(model2[2], 12))+"x^2 + "+str(round(model2[1], 5))+"x + "+str(round(model2[0], 4))
            y2_r2 = r2_score(dt, model2(_t))

            axs[i].plot(water['Time'], water['TempC'], label='Water Temp', linewidth=1.5, color="#013ADB")
            axs[i].plot(tt, dt, label=TAGS[i], linewidth=1.0, color="#FF0000")
            axs[i].plot(tt, model2(_t), label='Quadratic Fit, R^2='+str(round(y2_r2, 4))+strmodel2, linewidth=1.5, color="#01F208")

            axs[i].set_ylabel('Temperature (C)', fontsize=14)
            axs[i].set_xlabel('Time (s)', fontsize=14)
            axs[i].set_title('Temp v Time, ' + TAGS[i] + ", RSSI=10", fontsize=14)
            axs[i].legend()

fig.subplots_adjust(hspace=0.3)